In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_youxi_GMV=pd.DataFrame(columns=columns)
channel_youxi_GMV

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [6]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[6]/div/a/span').click()  #选择频道

In [5]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[6]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'游戏'

In [7]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[7]/a').click()  #选择子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[7]/a').text
sub_channel

'GMV'

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [9]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_youxi_GMV.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 
    


2020-12-16 00:12:46.346722


<ipython-input-9-b29857af812a>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-b29857af812a>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-b29857af812a>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1pJ411J76A视频属性的抓取
BV号为1pJ411J76A的视频属性抓取结束


<ipython-input-9-b29857af812a>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1e7411v72D视频属性的抓取
BV号为1e7411v72D的视频属性抓取结束
开始BV号为1e7411i7QG视频属性的抓取
BV号为1e7411i7QG的视频属性抓取结束
开始BV号为1vJ411E7qQ视频属性的抓取
BV号为1vJ411E7qQ的视频属性抓取结束
开始BV号为1kJ411n7tH视频属性的抓取
BV号为1kJ411n7tH的视频属性抓取结束
开始BV号为1P7411e7K4视频属性的抓取
BV号为1P7411e7K4的视频属性抓取结束
开始BV号为1v7411e7QV视频属性的抓取
BV号为1v7411e7QV的视频属性抓取结束
开始BV号为1F7411v7Bg视频属性的抓取
BV号为1F7411v7Bg的视频属性抓取结束
开始BV号为1mJ411b7yc视频属性的抓取
BV号为1mJ411b7yc的视频属性抓取结束
开始BV号为14J411V7qW视频属性的抓取
BV号为14J411V7qW的视频属性抓取结束
开始BV号为1bJ41177fE视频属性的抓取
BV号为1bJ41177fE的视频属性抓取结束
开始BV号为1Q7411i7wM视频属性的抓取
BV号为1Q7411i7wM的视频属性抓取结束
开始BV号为1f7411a7mz视频属性的抓取
BV号为1f7411a7mz的视频属性抓取结束
开始BV号为1Z7411e7sN视频属性的抓取
BV号为1Z7411e7sN的视频属性抓取结束
开始BV号为1M7411k7g2视频属性的抓取
BV号为1M7411k7g2的视频属性抓取结束
开始BV号为1j7411v7B7视频属性的抓取
BV号为1j7411v7B7的视频属性抓取结束
开始BV号为14J411771r视频属性的抓取
BV号为14J411771r的视频属性抓取结束
开始BV号为1aJ4117772视频属性的抓取
BV号为1aJ4117772的视频属性抓取结束
开始BV号为1t7411a7D7视频属性的抓取
BV号为1t7411a7D7的视频属性抓取结束
开始BV号为1bJ41187YQ视频属性的抓取
BV号为1bJ41187YQ的视频属性抓取结束
开始BV号为1a7411r7au视频属性的抓取
BV号为1a7411r7au的视频属性抓取结束
开始BV号为1E7411e7sQ视频属性的抓取
BV号为1E7411e7sQ的视

...

<ipython-input-9-b29857af812a>:187: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1Ye411p7YF视频属性的抓取
开始BV号为1Ye411p7YF视频属性的抓取
BV号为1Ye411p7YF的视频属性抓取结束
开始BV号为1BC4y1p71G视频属性的抓取
BV号为1BC4y1p71G的视频属性抓取结束
开始BV号为1Qf4y117e2视频属性的抓取
BV号为1Qf4y117e2的视频属性抓取结束
开始BV号为18t4y117EG视频属性的抓取
BV号为18t4y117EG的视频属性抓取结束
开始BV号为1mK4y1b79X视频属性的抓取
BV号为1mK4y1b79X的视频属性抓取结束
开始BV号为1NK4y187RE视频属性的抓取
BV号为1NK4y187RE的视频属性抓取结束
开始BV号为11t4y1C7u2视频属性的抓取
BV号为11t4y1C7u2的视频属性抓取结束
开始BV号为1H541147ub视频属性的抓取
BV号为1H541147ub的视频属性抓取结束
开始BV号为1op4y1Q7P1视频属性的抓取
BV号为1op4y1Q7P1的视频属性抓取结束
开始BV号为1iT4y137SR视频属性的抓取
BV号为1iT4y137SR的视频属性抓取结束
开始BV号为1Ng4y167HJ视频属性的抓取
BV号为1Ng4y167HJ的视频属性抓取结束
开始BV号为1Pk4y1r7P4视频属性的抓取
BV号为1Pk4y1r7P4的视频属性抓取结束
开始BV号为1Az411q7dJ视频属性的抓取
BV号为1Az411q7dJ的视频属性抓取结束
开始BV号为17z4y197hA视频属性的抓取
BV号为17z4y197hA的视频属性抓取结束
开始BV号为1pe411W7ty视频属性的抓取
BV号为1pe411W7ty的视频属性抓取结束
开始BV号为1Eg4y1B7yH视频属性的抓取
BV号为1Eg4y1B7yH的视频属性抓取结束
开始BV号为1be411W7Dp视频属性的抓取
BV号为1be411W7Dp的视频属性抓取结束
开始BV号为1tg4y1i7wb视频属性的抓取
BV号为1tg4y1i7wb的视频属性抓取结束
开始BV号为1jK411W7CY视频属性的抓取
BV号为1jK411W7CY的视频属性抓取结束
开始BV号为1KQ4y1K7qg视频属性的抓取
BV号为1KQ4y1K7qg的视频属性抓取结束
开始BV号为1654y1D7oC

BV号为1j54y1B7dp的视频属性抓取结束
开始BV号为1R54y1B75c视频属性的抓取
BV号为1R54y1B75c的视频属性抓取结束
开始BV号为1Jv411B7JU视频属性的抓取
BV号为1Jv411B7JU的视频属性抓取结束
开始BV号为1ai4y1G7df视频属性的抓取
BV号为1ai4y1G7df的视频属性抓取结束
开始BV号为1wC4y1Y7ay视频属性的抓取
BV号为1wC4y1Y7ay的视频属性抓取结束
开始BV号为1KC4y1Y7Nu视频属性的抓取
BV号为1KC4y1Y7Nu的视频属性抓取结束
开始BV号为1Et4y1978V视频属性的抓取
BV号为1Et4y1978V的视频属性抓取结束
开始BV号为1k54y1B7bv视频属性的抓取
BV号为1k54y1B7bv的视频属性抓取结束
开始BV号为1dA411q75x视频属性的抓取
BV号为1dA411q75x的视频属性抓取结束
开始BV号为1Pg4y1v7wL视频属性的抓取
BV号为1Pg4y1v7wL的视频属性抓取结束
开始BV号为1CZ4y1H7So视频属性的抓取
BV号为1CZ4y1H7So的视频属性抓取结束
开始BV号为18Z4y1u7xA视频属性的抓取
BV号为18Z4y1u7xA的视频属性抓取结束
开始BV号为1ui4y137Wq视频属性的抓取
BV号为1ui4y137Wq的视频属性抓取结束
开始BV号为1Xa4y1h7LT视频属性的抓取
BV号为1Xa4y1h7LT的视频属性抓取结束
开始BV号为1Rt4y197cH视频属性的抓取
BV号为1Rt4y197cH的视频属性抓取结束
开始BV号为14a4y1a7gF视频属性的抓取
BV号为14a4y1a7gF的视频属性抓取结束
开始BV号为1of4y1R7bB视频属性的抓取
BV号为1of4y1R7bB的视频属性抓取结束
开始BV号为1Lp4y1S7XN视频属性的抓取
BV号为1Lp4y1S7XN的视频属性抓取结束
开始BV号为1Rp4y1U73K视频属性的抓取
BV号为1Rp4y1U73K的视频属性抓取结束
开始BV号为1MC4y1b72R视频属性的抓取
BV号为1MC4y1b72R的视频属性抓取结束
开始BV号为1S5411Y7M4视频属性的抓取
BV号为1S5411Y7M4的视频属性抓取结束
开始BV号为1FV41167ou

BV号为1PC4y1t7Xw的视频属性抓取结束
开始BV号为1Qk4y117Kk视频属性的抓取
BV号为1Qk4y117Kk的视频属性抓取结束
开始BV号为11D4y1U7w6视频属性的抓取
BV号为11D4y1U7w6的视频属性抓取结束
开始BV号为1M54y1e7QD视频属性的抓取
BV号为1M54y1e7QD的视频属性抓取结束
开始BV号为1Zt4y1D7LL视频属性的抓取
BV号为1Zt4y1D7LL的视频属性抓取结束
开始BV号为1kK4y1v7x4视频属性的抓取
BV号为1kK4y1v7x4的视频属性抓取结束
开始BV号为15D4y1m7gr视频属性的抓取
BV号为15D4y1m7gr的视频属性抓取结束
开始BV号为1hD4y1m7wG视频属性的抓取
BV号为1hD4y1m7wG的视频属性抓取结束
开始BV号为1wi4y1g7ca视频属性的抓取
BV号为1wi4y1g7ca的视频属性抓取结束
开始BV号为1464y1c7rF视频属性的抓取
BV号为1464y1c7rF的视频属性抓取结束
开始BV号为1at4y1X7gT视频属性的抓取
BV号为1at4y1X7gT的视频属性抓取结束
开始BV号为1Fp4y1i7Ma视频属性的抓取
BV号为1Fp4y1i7Ma的视频属性抓取结束
开始BV号为1x54y1i7dN视频属性的抓取
BV号为1x54y1i7dN的视频属性抓取结束
开始BV号为1X64y1c7vo视频属性的抓取
BV号为1X64y1c7vo的视频属性抓取结束
开始BV号为11T4y157tQ视频属性的抓取
BV号为11T4y157tQ的视频属性抓取结束
开始BV号为1tD4y1m7T7视频属性的抓取
BV号为1tD4y1m7T7的视频属性抓取结束
开始BV号为1oT4y1j78m视频属性的抓取
BV号为1oT4y1j78m的视频属性抓取结束
开始BV号为1eV411U7nc视频属性的抓取
BV号为1eV411U7nc的视频属性抓取结束
开始BV号为19t4y1D794视频属性的抓取
BV号为19t4y1D794的视频属性抓取结束
开始BV号为1et4y1D7de视频属性的抓取
BV号为1et4y1D7de的视频属性抓取结束
开始BV号为1it4y1D7eJ视频属性的抓取
BV号为1it4y1D7eJ的视频属性抓取结束
开始BV号为17A411n7Ya

开始BV号为1Kr4y1w7gR视频属性的抓取
BV号为1Kr4y1w7gR的视频属性抓取结束
开始BV号为1dp4y1a7S1视频属性的抓取
BV号为1dp4y1a7S1的视频属性抓取结束
开始BV号为1ci4y177kV视频属性的抓取
BV号为1ci4y177kV的视频属性抓取结束
开始BV号为1xT4y1F7dk视频属性的抓取
BV号为1xT4y1F7dk的视频属性抓取结束
开始BV号为1AK411P7gY视频属性的抓取
BV号为1AK411P7gY的视频属性抓取结束
开始BV号为1iV411y7YK视频属性的抓取
BV号为1iV411y7YK的视频属性抓取结束
开始BV号为1354y127xZ视频属性的抓取
BV号为1354y127xZ的视频属性抓取结束
开始BV号为1AA411j7e3视频属性的抓取
BV号为1AA411j7e3的视频属性抓取结束
开始BV号为1bf4y1B7uV视频属性的抓取
BV号为1bf4y1B7uV的视频属性抓取结束
开始BV号为1iT4y1c7Xx视频属性的抓取
BV号为1iT4y1c7Xx的视频属性抓取结束
开始BV号为16K4y177uv视频属性的抓取
BV号为16K4y177uv的视频属性抓取结束
开始BV号为1jD4y1974F视频属性的抓取
BV号为1jD4y1974F的视频属性抓取结束
开始BV号为1th41197iw视频属性的抓取
BV号为1th41197iw的视频属性抓取结束
开始BV号为1Ey4y1B7Wc视频属性的抓取
BV号为1Ey4y1B7Wc的视频属性抓取结束
开始BV号为1ST4y1c7Hb视频属性的抓取
BV号为1ST4y1c7Hb的视频属性抓取结束
开始BV号为1s54y1k7Pw视频属性的抓取
BV号为1s54y1k7Pw的视频属性抓取结束
开始BV号为1mz4y1o7M4视频属性的抓取
BV号为1mz4y1o7M4的视频属性抓取结束
开始BV号为1nK411N7pm视频属性的抓取
BV号为1nK411N7pm的视频属性抓取结束
开始BV号为1D541157MB视频属性的抓取
BV号为1D541157MB的视频属性抓取结束
开始BV号为1y54y1y7mC视频属性的抓取
BV号为1y54y1y7mC的视频属性抓取结束
开始BV号为1qz4y1d7Dp视频属性的抓取
BV号为1qz4y1d7Dp的视

In [10]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-10-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [10]:
channel_youxi_GMV.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [11]:
channel_youxi_GMV.to_json('channel_youxi_GMV.json')

In [17]:
pd.set_option('display.max_rows', None)

In [18]:
channel_youxi_danji[channel_youxi_danji['时间']=='4月']

,时间,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1h54y1d7Qq,4月,鹤吱菌,个人认证,191.3万,81,【师生对线】这网课上得就尼玛离谱！（第五集）,游戏,单机游戏,活动作品,自制,"[配音, 守望先锋, 独立游戏, 搞笑, 单机游戏, 游戏, 单机游戏, 电子竞技, 配音大...",13:00,953.7万,114.2万,115.2万,42.5万,8.9万,25302,18.8万,否
1L64y1M7Dp,4月,老番茄,个人认证,1360.1万,348,【老番茄】史上最骚骑士,游戏,单机游戏,活动作品,自制,"[骑马与砍杀, 搞笑, 游戏, 单机游戏, 老番茄, 骑马与砍杀2, 游戏练级挑战, 骑砍,...",15:45,902.3万,84.5万,61.7万,24.5万,2.4万,21336,9.7万,否
18K41157Tu,4月,老番茄,个人认证,1360.1万,348,宫斗大师老番茄,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 宫妃, 易次元]",13:55,739.6万,45.1万,9.3万,5.5万,1.3万,14189,10.6万,否
1PT4y1G7ME,4月,老番茄,个人认证,1360.1万,348,常胜将军老番茄,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 老番茄, shield wall]",14:54,726.8万,48.7万,12.9万,5.8万,7401,15180,7.0万,否
1S54y197uX,4月,徐大虾咯,个人认证,570.1万,224,家 里 有 矿 ！,游戏,单机游戏,普通视频,自制,"[我的世界, 单机游戏, 游戏, 单机游戏, 徐大虾咯, 徐大虾, MINECRAFT, M...",12:18,714.5万,35.0万,8.8万,4.3万,4860,7738,6.7万,否
1ti4y1t7YM,4月,老番茄,个人认证,1360.1万,348,解说挑战！什么游戏都能解说？,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 老番茄]",08:19,711.2万,61.4万,16.7万,8.1万,9970,14368,10.6万,否
1Ai4y1b7ET,4月,敖厂长,个人认证,735.1万,348,花了一万多元终于见到最早开放世界游戏之一,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 游戏资讯, 敖厂长, 单机, 游戏杂谈, 游戏视频, 杂谈]",14:26,670.4万,112.6万,156.8万,34.9万,2.4万,15618,8.6万,否
1Up4y1C7PB,4月,丢鹏D1uuu,无,8.4万,20,年级第一：别TM挡着我上网课！！！,游戏,单机游戏,活动作品,自制,"[独立游戏, 搞笑, 单机游戏, 游戏, 单机游戏, bilibili新星计划, 新人, 恶...",05:38,654.0万,17.3万,1.5万,2.0万,1321,1952,4526,否
1wi4y187bu,4月,中国BOY超级大猩猩,个人认证,663.5万,1380,全员铁憨憨!,游戏,单机游戏,活动作品,自制,"[STEAM, 单机游戏, 游戏, 单机游戏, 花少北, 老番茄, 中国BOY, LexBu...",19:57,653.2万,42.0万,23.9万,14.2万,2.5万,13493,9.7万,否
13t4y1U7kQ,4月,徐大虾咯,个人认证,570.1万,224,德华的世界#2,游戏,单机游戏,普通视频,自制,"[我的世界, 单机游戏, 游戏, 单机游戏, 徐大虾咯, 徐大虾, MINECRAFT, M...",12:07,619.7万,35.1万,9.5万,4.4万,7033,7937,6.3万,否
